Using improper features here for a multiple regression analysis. Will re-configure and optimize.

## Question
What impact does a country's economic status (as measured by GDP and other factors) affect the number of human trafficking cases reported in that country?

### Data
The base dataset includes features for:

* Year
* Data Source
* Gender
* Citizenship
* Victim Forced to Labor
* Type of Control Exerted Upon Victim
* Recruiter Category
* Exploit Type
* Labor Type
* Age Categories
* GDP
* Unemployment Rate

*Not all of these features will be used in the model we create to predict the answers to our question.*

### Model
Given an input set, we are trying to predict an outcome. In our base set, we have a known outcome (the type of labor forced upon the victim (represented by the 'Labor_Type' feature)). Can we use multiple regression in a supervised model to create predictions.

### Output
We hope to find a line where as GDP, unemployment and other factors change, the number of human trafficking cases may be predicted. If the number of cases can be predicted/modeled, one of two outcomes may be possible:
* Examine existing cases for possible under-reporting and encouraging law enforcement to act more pro-actively.
* Allow leadership in given countries the opportunity to curb trafficking by improving economic conditions within their respective country.

In [1]:
# Import Dependencies
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from config import conn

# from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [2]:
# Get trafficking data
trafficking_df = pd.read_sql_table("Trafficking_Cleaned", conn)
trafficking_df

,yearOfRegistration,Datasource,gender,citizenship,isForcedLabour,ControlCategory,RecruiterCategory,ExploitType,Labor_Type,ageCategories
0,2012,Case Management,Female,LK,1,Threats,Other,Forced Labor,Domestic Work,Age 30-38
1,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38
2,2012,Case Management,Female,LK,1,Threats,Other,Forced Labor,Domestic Work,Age 30-38
3,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38
4,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38
...,...,...,...,...,...,...,...,...,...,...
14294,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17
14295,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17
14296,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17
14297,2018,Hotline,Male,US,0,Other,Family/Relative,Sexual Exploitation,Unknown,Age 9-17


In [3]:
# Get unemployment data
unemployment_df = pd.read_sql_table("WEO_Unemployment", conn, index_col=["ISO"])
unemployment_df

,WEO Subject Code,Country,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
ISO,,,,,,,,,,,,,,,,,,,
AFG,LUR,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALB,LUR,Albania,15.751,15.000,14.400,14.100,13.800,13.400,13.100,13.800,14.000,14.000,13.400,15.900,17.5,17.100,15.200,13.700,12.300
DZA,LUR,Algeria,25.664,23.716,17.656,15.265,12.512,13.793,11.343,10.167,9.961,9.971,10.969,9.829,10.6,11.214,10.498,11.709,11.731
AGO,LUR,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATG,LUR,Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VNM,LUR,Vietnam,6.010,5.780,5.600,5.310,4.820,4.640,2.380,2.900,2.880,2.220,1.960,2.180,2.1,2.330,2.300,2.240,2.190
WBG,LUR,West Bank and Gaza,31.200,25.500,26.800,23.500,23.700,21.700,26.600,24.500,23.700,20.900,23.000,23.400,26.9,25.900,26.900,25.450,26.250
YEM,LUR,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Get GDP data
gdp_df = pd.read_sql_table("WEO_GDP", conn, index_col=["ISO"])
gdp_df

,WEO Subject Code,Country,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
ISO,,,,,,,,,,,,,,,,,,,
AFG,NGDPDPC,Afghanistan,233.433,233.755,254.259,294.396,320.674,381.502,447.746,511.374,631.490,714.70,784.611,754.402,747.622,711.337,616.234,636.693,582.323
ALB,NGDPDPC,Albania,1425.130,1846.120,2373.580,2673.770,2972.750,3595.050,4370.560,4114.090,4097.830,4439.89,4248.910,4415.600,4584.920,3953.610,4124.410,4542.760,5284.440
DZA,NGDPDPC,Algeria,1806.860,2128.390,2636.310,3141.030,3508.960,3986.560,4943.500,3886.060,4480.720,5453.89,5575.650,5477.060,5466.330,4153.320,3918.940,4079.650,4118.740
AGO,NGDPDPC,Angola,841.420,951.969,1222.040,1862.420,2561.860,3099.090,4081.690,3146.800,3641.440,4716.25,5245.020,5436.520,5625.740,4354.920,3676.830,4303.690,3620.590
ATG,NGDPDPC,Antigua and Barbuda,10482.010,10892.400,11581.320,12736.240,14262.100,15992.740,16510.080,14612.900,13564.680,13295.36,13811.080,13392.220,13951.990,14696.890,15556.010,15655.230,16860.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VNM,NGDPDPC,Vietnam,546.555,610.357,756.981,873.136,996.255,1152.300,1446.770,1481.810,1628.520,1950.46,2197.610,2370.090,2566.910,2582.390,2720.190,2957.930,3211.460
WBG,NGDPDPC,West Bank and Gaza,1102.500,1197.160,1350.910,1461.090,1480.700,1563.700,1913.320,2061.560,2406.260,2711.92,2888.600,3122.990,3158.600,3084.130,3325.850,3407.310,3353.230
YEM,NGDPDPC,Yemen,560.031,597.771,682.148,797.704,881.648,971.326,1171.170,1060.930,1266.790,1302.30,1367.720,1515.950,1574.250,1500.690,1061.840,891.888,762.136


In [11]:
# Get poverty data
poverty_df = pd.read_sql_table("WorldBank_Poverty", conn, index_col=["Country Code"])
poverty_df

,Indicator Code,Country Name,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Code,,,,,,,,,,,,,,,,,,,
ABW,SI.POV.NAHC,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFE,SI.POV.NAHC,Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,SI.POV.NAHC,Afghanistan,NaN,NaN,NaN,NaN,NaN,33.7,NaN,NaN,NaN,38.3,NaN,NaN,NaN,NaN,54.5,NaN,NaN
AFW,SI.POV.NAHC,Africa Western and Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AGO,SI.POV.NAHC,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,SI.POV.NAHC,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.9,17.6,21.1,17.6,NaN,NaN,NaN
YEM,SI.POV.NAHC,"Yemen, Rep.",NaN,NaN,NaN,34.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.6,NaN,NaN,NaN,NaN
ZAF,SI.POV.NAHC,South Africa,NaN,NaN,NaN,66.6,NaN,NaN,62.1,NaN,53.2,NaN,NaN,NaN,55.5,NaN,NaN,NaN,NaN


In [7]:
# Get country data
countries_df = pd.read_sql_table("Country_Codes", conn)
countries_df

,country_name,country_iso2,country_iso3,country_un_code
0,Afghanistan,AF,AFG,4
1,Aland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16
...,...,...,...,...
242,Wallis and Futuna Islands,WF,WLF,876
243,Western Sahara,EH,ESH,732
244,Yemen,YE,YEM,887
245,Zambia,ZM,ZMB,894


In [8]:
trafficking_x_countries = trafficking_df.merge(countries_df, how='left', left_on='citizenship', right_on='country_iso2')
trafficking_x_countries

,yearOfRegistration,Datasource,gender,citizenship,isForcedLabour,ControlCategory,RecruiterCategory,ExploitType,Labor_Type,ageCategories,country_name,country_iso2,country_iso3,country_un_code
0,2012,Case Management,Female,LK,1,Threats,Other,Forced Labor,Domestic Work,Age 30-38,Sri Lanka,LK,LKA,144
1,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38,Sri Lanka,LK,LKA,144
2,2012,Case Management,Female,LK,1,Threats,Other,Forced Labor,Domestic Work,Age 30-38,Sri Lanka,LK,LKA,144
3,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38,Sri Lanka,LK,LKA,144
4,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38,Sri Lanka,LK,LKA,144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14294,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17,United States of America,US,USA,840
14295,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17,United States of America,US,USA,840
14296,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17,United States of America,US,USA,840
14297,2018,Hotline,Male,US,0,Other,Family/Relative,Sexual Exploitation,Unknown,Age 9-17,United States of America,US,USA,840


In [9]:
# Get total case counts for each country by year
cases_by_year = trafficking_x_countries.groupby(["country_iso3", "yearOfRegistration"]).size().reset_index(name="case_counts")
cases_by_year

,country_iso3,yearOfRegistration,case_counts
0,AFG,2013,3
1,AFG,2014,1
2,BLR,2015,90
3,BLR,2016,112
4,BLR,2017,30
5,CHN,2017,11
6,CHN,2018,12
7,ERI,2014,1
8,HTI,2012,89
9,IDN,2014,1


In [12]:
# Get unemployment and GDP and poverty rate by year and by country for each entry in our cumulative DataFrame
year_unemployment = []
year_gdp = []
year_poverty = []

for row in cases_by_year.iterrows():
    year_unemployment.append(unemployment_df.loc[row[1]['country_iso3'], str(row[1]['yearOfRegistration'])])
    year_gdp.append(gdp_df.loc[row[1]['country_iso3'], str(row[1]['yearOfRegistration'])])
    year_poverty.append(poverty_df.loc[row[1]['country_iso3'], str(row[1]['yearOfRegistration'])])

# Add our results to our cumulative DataFrame
cases_by_year['Unemployment'] = year_unemployment
cases_by_year['GDP'] = year_gdp
cases_by_year['PovertyRate'] = year_poverty

In [13]:
# Look at our DataFrame
cases_by_year

,country_iso3,yearOfRegistration,case_counts,Unemployment,GDP,PovertyRate
0,AFG,2013,3,NaN,754.402,NaN
1,AFG,2014,1,NaN,747.622,NaN
2,BLR,2015,90,5.821,5941.240,5.1
3,BLR,2016,112,5.920,5022.470,5.7
4,BLR,2017,30,5.684,5757.290,5.9
5,CHN,2017,11,3.900,8823.460,3.1
6,CHN,2018,12,3.800,9919.810,1.7
7,ERI,2014,1,NaN,786.540,NaN
8,HTI,2012,89,NaN,1337.340,58.5
9,IDN,2014,1,5.940,3533.610,11.3


In [14]:
# Drop rows with null values even though it's going to limit our dataset
cases_by_year.dropna(inplace=True)
cases_by_year

,country_iso3,yearOfRegistration,case_counts,Unemployment,GDP,PovertyRate
2,BLR,2015,90,5.821,5941.24,5.1
3,BLR,2016,112,5.920,5022.47,5.7
4,BLR,2017,30,5.684,5757.29,5.9
5,CHN,2017,11,3.900,8823.46,3.1
6,CHN,2018,12,3.800,9919.81,1.7
9,IDN,2014,1,5.940,3533.61,11.3
10,IDN,2015,6,6.180,3367.69,11.2
11,IDN,2016,28,5.610,3605.72,10.9
12,IDN,2017,34,5.500,3885.47,10.6
14,KGZ,2014,71,8.046,1292.62,30.6


In [29]:
year_series = cases_by_year['yearOfRegistration']
year_series = year_series.astype(str)
cases_by_year['yearString'] = year_series
cases_by_year.drop(columns=['yearOfRegistration'], axis=1, inplace=True)
categorical_features = cases_by_year.dtypes[cases_by_year.dtypes == 'object'].index.tolist()
categorical_features

['country_iso3', 'yearString']

In [30]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder Instance
cat_encoder = OneHotEncoder(sparse=False)

# Fit and transform using category variable list
encoded_df = pd.DataFrame(cat_encoder.fit_transform(cases_by_year[categorical_features]))

# Add encoded variables to DataFrame
encoded_df.columns = cat_encoder.get_feature_names(categorical_features)
encoded_df

,country_iso3_BLR,country_iso3_CHN,country_iso3_IDN,country_iso3_KGZ,country_iso3_LKA,country_iso3_PHL,country_iso3_THA,country_iso3_UKR,yearString_2012,yearString_2013,yearString_2014,yearString_2015,yearString_2016,yearString_2017,yearString_2018
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [35]:
# Merge to original and drop
coded_cases_by_year = cases_by_year.merge(encoded_df, left_index=True, right_index=True)
coded_cases_by_year.drop(categorical_features, axis=1, inplace=True)
coded_cases_by_year

,case_counts,Unemployment,GDP,PovertyRate,yearCategory,country_iso3_BLR,country_iso3_CHN,country_iso3_IDN,country_iso3_KGZ,country_iso3_LKA,country_iso3_PHL,country_iso3_THA,country_iso3_UKR,yearString_2012,yearString_2013,yearString_2014,yearString_2015,yearString_2016,yearString_2017,yearString_2018
2,90,5.821,5941.24,5.1,2015,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,112,5.920,5022.47,5.7,2016,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,30,5.684,5757.29,5.9,2017,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,11,3.900,8823.46,3.1,2017,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,12,3.800,9919.81,1.7,2018,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,1,5.940,3533.61,11.3,2014,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10,6,6.180,3367.69,11.2,2015,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
11,28,5.610,3605.72,10.9,2016,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12,34,5.500,3885.47,10.6,2017,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
14,71,8.046,1292.62,30.6,2014,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [36]:
coded_cases_by_year.dtypes

case_counts            int64
Unemployment         float64
GDP                  float64
PovertyRate          float64
yearCategory        category
country_iso3_BLR     float64
country_iso3_CHN     float64
country_iso3_IDN     float64
country_iso3_KGZ     float64
country_iso3_LKA     float64
country_iso3_PHL     float64
country_iso3_THA     float64
country_iso3_UKR     float64
yearString_2012      float64
yearString_2013      float64
yearString_2014      float64
yearString_2015      float64
yearString_2016      float64
yearString_2017      float64
yearString_2018      float64
dtype: object

In [37]:
coded_cases_by_year.drop(columns=['yearCategory'], axis=1, inplace=True)
coded_cases_by_year.dtypes

case_counts           int64
Unemployment        float64
GDP                 float64
PovertyRate         float64
country_iso3_BLR    float64
country_iso3_CHN    float64
country_iso3_IDN    float64
country_iso3_KGZ    float64
country_iso3_LKA    float64
country_iso3_PHL    float64
country_iso3_THA    float64
country_iso3_UKR    float64
yearString_2012     float64
yearString_2013     float64
yearString_2014     float64
yearString_2015     float64
yearString_2016     float64
yearString_2017     float64
yearString_2018     float64
dtype: object

In [41]:
# Split processed data in to targets and features
y = coded_cases_by_year["case_counts"]
X = coded_cases_by_year.drop(columns=["case_counts"])

# Split data into a training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 52)

In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [49]:
# Create deep neural net model
input_feature_number = len(X_train_scaled[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 12

tf_nn = tf.keras.models.Sequential()

# 1st hidden layer
tf_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_feature_number, activation="relu"))

# 2nd hidden layer
tf_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
tf_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Model structure
tf_nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 20)                380       
_________________________________________________________________
dense_13 (Dense)             (None, 12)                252       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 13        
Total params: 645
Trainable params: 645
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Compile model
tf_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model
fit_model = tf_nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 503ms/step - loss: -16.6777 - accuracy: 0.1111
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: -17.9073 - accuracy: 0.1111
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: -19.1360 - accuracy: 0.1111
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: -20.3677 - accuracy: 0.1111
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: -21.6041 - accuracy: 0.1111
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: -22.8449 - accuracy: 0.1111
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: -24.0910 - accuracy: 0.1111
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: -25.3427 - accuracy: 0.1111
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: -26.5776 - accuracy: 0.1111
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: -27.6917 - accuracy: 0.1111
Epoch 1